In [8]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('Scripts')

from SignalDataset import SignalDataset
from Dataloader import Dataloader
from SSVEPDataset import SSVEPSignalDataset
from SSVEPDataloader import SSVEPDataloaders
from utils import generate_time_series
from dataprep import dataprep
from SignalDatasetCreator import SignalDatasetCreator
from torch.optim.lr_scheduler import _LRScheduler

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from TrainingFunc import train, test

ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
dataprep('768')

In [95]:
SignalDatasetCreator(512)

In [3]:
dataset = SignalDataset()
dataloaders, dataset_sizes = Dataloader(dataset, batch_size = 128)

In [4]:
dataType = '512'
data = pd.read_csv('SSVEPDatasetPCA_'+dataType+'.csv')
ssvep_dataset = SSVEPSignalDataset(data)
ssvep_dataloaders, ssvep_datasets_izes = Dataloader(ssvep_dataset, batch_size = 16)

In [9]:
class CyclicLR(_LRScheduler):
    def __init__(self, optimizer, schedule, last_epoch = -1):
        assert callable(scheduler)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)
        
    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]
    
def cosine(t_max, eta_min = 0):
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim  = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
        
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]
    

input_dim = 512
hidden_dim = 1024
layer_dim = 3
output_dim = 11
seq_dim = 128
model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim).to(device)

In [12]:
exemplo.shape

torch.Size([128, 512])

In [10]:
exemplo = iter(dataloaders['train']).next()['signal'].to(device)
model(exemplo)

RuntimeError: input must have 3 dimensions, got 2

In [109]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.Linear1 = nn.Linear(512, 5000)
        self.Linear4 = nn.Linear(5000, 11)
        
    def forward(self, X):
        X = self.Linear1(X)
#         X = self.Linear2(X)
#         X = self.Linear3(X)
        X = self.Linear4(X)
        return X

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
model = Model().double().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)
criterion = nn.CrossEntropyLoss()

In [110]:
model, training_stats, validation_stats = train(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, device,
                                                num_epochs = 2)
model, training_stats, validation_stats = train(model, ssvep_dataloaders, ssvep_datasets_izes, criterion, optimizer, scheduler, device,
                                               num_epochs = 10)

Epoch 0/1
----------
train Loss: 0.0714 Acc: 0.9773
val Loss: 0.0004 Acc: 1.0000

Epoch 1/1
----------
train Loss: 0.0000 Acc: 1.0000
val Loss: 0.0000 Acc: 1.0000

Training complete in 0m 4s
Best val Acc: 1.000000
Epoch 0/9
----------
train Loss: 3.5437 Acc: 0.1938
val Loss: 3.9050 Acc: 0.1543

Epoch 1/9
----------
train Loss: 3.3212 Acc: 0.2369
val Loss: 3.7572 Acc: 0.1975

Epoch 2/9
----------
train Loss: 3.0794 Acc: 0.2646
val Loss: 3.4364 Acc: 0.2222

Epoch 3/9
----------
train Loss: 2.8091 Acc: 0.3000
val Loss: 3.0908 Acc: 0.2099

Epoch 4/9
----------
train Loss: 2.9027 Acc: 0.2908
val Loss: 4.7213 Acc: 0.1975

Epoch 5/9
----------
train Loss: 2.0935 Acc: 0.4200
val Loss: 3.2112 Acc: 0.2346

Epoch 6/9
----------
train Loss: 1.2605 Acc: 0.5400
val Loss: 2.8530 Acc: 0.2160

Epoch 7/9
----------
train Loss: 1.1248 Acc: 0.5446
val Loss: 2.8288 Acc: 0.2037

Epoch 8/9
----------
train Loss: 1.0853 Acc: 0.5800
val Loss: 2.9098 Acc: 0.1975

Epoch 9/9
----------
train Loss: 1.0586 Acc: 0.5

In [124]:
test(model, dataloaders['test'], dataset_sizes['test'], criterion, device)

path = 'C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\Models\\SignalSelector'
torch.save(model.state_dict(), path+'\\selector.pth')

Test Loss: 0.0000 Acc: 1.0000


In [56]:
path = 'C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\Backup\\ProcessedSignals\\512'
ssvep_dataloaders, _ = Dataloader(ssvep_dataset, batch_size = 1)
alpha = 10
i = [-1, -1, -1, -1, -1]
data = pd.DataFrame(columns = ['subject', 'state', 'path'])
for t in ['train', 'val', 'test']:
    dataloader = ssvep_dataloaders[t]
    for sample in dataloader:
        c = sample['class'].item()
        i[c] += 1
        state = c + 1
        new_path = path+'\\'+str(state)

        signal = sample['signal'].squeeze(0)

        _, frequency = torch.max(model(signal.to(device)), 0)
        frequency = frequency.item() + 10

        signal = signal.numpy() + generate_time_series(frequency = frequency)

        new_path = new_path+'\\'+str(i[c])+'.csv'
        np.savetxt(new_path, signal)
        
        temp = pd.DataFrame([['_', state, new_path]] , columns = ['subject', 'state', 'path'])
        data = pd.concat([data, temp], axis = 0)
        
data.to_csv('SSVEPDatasetProcessed_512.csv', index = False)

***

Gramian Angular Field

In [57]:
import sys
sys.path.append('Scripts')
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from utils import path, generate_time_series
from sklearn import preprocessing
import os
import gc

from mpl_toolkits.axes_grid1 import ImageGrid
from pyts.image import GramianAngularField

def read_data(dataframe):
    return pd.read_csv(dataframe['path'], names = range(1, 16+1), dtype = float)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
dataType = '512'
method = 'Simple'
methodDict = {
    'Simple': ['Dados_BCI_Combinados', 'GAF_Combinados', 'Combined_'],
    'PCA': ['Dados_BCI_CombinadosPCA', 'GAF_PCA', 'PCA_'],
    'Processed': ['ProcessedSignals', 'GAF_ProcessedSignals', 'Processed_']
}
dataPath = path('C:\\Users\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\Backup\\'+methodDict[method][0]+'\\'+dataType)
figPath = path('C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\Backup\\'+methodDict[method][1]+"\\"+dataType)

data = pd.read_csv('SSVEPDataset'+methodDict[method][2]+dataType+".csv")

gasf = GramianAngularField(image_size = 80, method = 'summation')
files = list(range(len(data)))
n = len(files)

for i in range(1000, 1015):
    sample = np.loadtxt(data.loc[i]['path']).reshape(1, -1)
    
    _, frequency = torch.max(model(torch.as_tensor(sample).to(device)), 1)
    frequency = frequency.item() + 10
    artificial_series = 10**-7*generate_time_series(frequency = frequency)
    
    if sample[0][1] != 0:
        X = gasf.fit_transform(sample)
        Y = gasf.fit_transform(artificial_series.reshape(1, -1))
        fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (12, 12))
        plt.axis('off')
        im = ax.imshow(X[0] + Y[0], cmap = 'rainbow', origin = 'lower')

        state = str(data.loc[i]['state'])
        subject = str(data.loc[i]['subject'])
#         subject = '0_'
        if data.loc[i]['path'][-6] == '\\':
            file = data.loc[i]['path'][-5]
        else:
            file = data.loc[i]['path'][-6:-4]

        try:
            fig.savefig(figPath()+'\\'+state+'\\'+subject+file+'_'+str(i)+".png", dpi = 200, bbox_inches = 'tight')
            plt.close()
        except:
            print("Error creating picture: {}".format(figPath()+'\\'+state+'\\0_'+file+".png"))